# Real-World Project 2: Browser History Manager

In [ ]:
class WebPage:
    def __init__(self, url, title, timestamp=None):
        self.url = url
        self.title = title
        self.timestamp = timestamp or datetime.now()
        self.visit_count = 1
    
    def __str__(self):
        return f"{self.title} ({self.url})"
    
    def __eq__(self, other):
        if isinstance(other, WebPage):
            return self.url == other.url
        return False


class BrowserHistory:
    def __init__(self, max_history=50):
        self.back_stack = StackUsingLinkedList()  # Stack for back navigation
        self.forward_stack = StackUsingLinkedList()  # Stack for forward navigation
        self.current_page = None
        self.max_history = max_history
        self.all_pages = {}  # Store all visited pages for stats
    
    def visit(self, url, title):
        """Visit a new page"""
        # Create page object
        page = WebPage(url, title)
        
        # If we're currently on a page, push it to back stack
        if self.current_page:
            self.back_stack.push(self.current_page)
            
            # Clear forward stack when visiting new page
            self.forward_stack = StackUsingLinkedList()
        
        # Set new current page
        self.current_page = page
        
        # Track in all pages
        if url in self.all_pages:
            self.all_pages[url].visit_count += 1
        else:
            self.all_pages[url] = page
        
        print(f"Visited: {page}")
        
        # Enforce max history limit
        if self.back_stack.size() > self.max_history:
            self._remove_oldest()
    
    def go_back(self):
        """Go to previous page"""
        if self.back_stack.is_empty():
            print("Cannot go back - no history")
            return None
        
        if self.current_page:
            self.forward_stack.push(self.current_page)
        
        self.current_page = self.back_stack.pop()
        print(f"Went back to: {self.current_page}")
        return self.current_page
    
    def go_forward(self):
        """Go forward to next page"""
        if self.forward_stack.is_empty():
            print("Cannot go forward")
            return None
        
        if self.current_page:
            self.back_stack.push(self.current_page)
        
        self.current_page = self.forward_stack.pop()
        print(f"Went forward to: {self.current_page}")
        return self.current_page
    
    def get_current_page(self):
        """Get current page"""
        return self.current_page
    
    def get_back_count(self):
        """Get number of pages in back history"""
        return self.back_stack.size()
    
    def get_forward_count(self):
        """Get number of pages in forward history"""
        return self.forward_stack.size()
    
    def clear_history(self):
        """Clear all history"""
        self.back_stack = StackUsingLinkedList()
        self.forward_stack = StackUsingLinkedList()
        print("History cleared")
    
    def get_most_visited(self, n=5):
        """Get n most visited pages"""
        pages = list(self.all_pages.values())
        pages.sort(key=lambda x: x.visit_count, reverse=True)
        return pages[:n]
    
    def search_history(self, keyword):
        """Search history by keyword"""
        results = []
        keyword = keyword.lower()
        
        # Search in current structure (this is inefficient for large history)
        # In real browser, you'd use a proper database
        
        # Check back stack
        temp_stack = StackUsingLinkedList()
        while not self.back_stack.is_empty():
            page = self.back_stack.pop()
            temp_stack.push(page)
            
            if (keyword in page.url.lower() or 
                keyword in page.title.lower()):
                results.append(page)
        
        # Restore back stack
        while not temp_stack.is_empty():
            self.back_stack.push(temp_stack.pop())
        
        # Check forward stack
        temp_stack = StackUsingLinkedList()
        while not self.forward_stack.is_empty():
            page = self.forward_stack.pop()
            temp_stack.push(page)
            
            if (keyword in page.url.lower() or 
                keyword in page.title.lower()):
                results.append(page)
        
        # Restore forward stack
        while not temp_stack.is_empty():
            self.forward_stack.push(temp_stack.pop())
        
        return results
    
    def _remove_oldest(self):
        """Remove oldest page from history (when limit reached)"""
        # This is simplified - in real browser you'd use more sophisticated logic
        if self.back_stack.size() > 0:
            # Remove from bottom of stack (requires temporary stack)
            temp_stack = StackUsingLinkedList()
            
            # Move all except last to temp stack
            while self.back_stack.size() > 1:
                temp_stack.push(self.back_stack.pop())
            
            # Discard the last (oldest)
            self.back_stack.pop()
            
            # Restore back stack
            while not temp_stack.is_empty():
                self.back_stack.push(temp_stack.pop())
    
    def display_history(self):
        """Display browsing history"""
        print("\n" + "="*60)
        print("BROWSER HISTORY")
        print("="*60)
        
        print(f"Current page: {self.current_page}")
        
        print(f"\nBack History ({self.get_back_count()} pages):")
        print("-" * 40)
        self._display_stack(self.back_stack)
        
        print(f"\nForward History ({self.get_forward_count()} pages):")
        print("-" * 40)
        self._display_stack(self.forward_stack)
        
        # Most visited
        most_visited = self.get_most_visited(3)
        if most_visited:
            print(f"\nMost visited pages:")
            for i, page in enumerate(most_visited, 1):
                print(f"  {i}. {page} (visited {page.visit_count} times)")
    
    def _display_stack(self, stack):
        """Helper to display stack contents"""
        if stack.is_empty():
            print("  (empty)")
            return
        
        # Display without destroying stack
        temp_stack = StackUsingLinkedList()
        index = 1
        
        while not stack.is_empty():
            page = stack.pop()
            print(f"  {index}. {page}")
            temp_stack.push(page)
            index += 1
        
        # Restore stack
        while not temp_stack.is_empty():
            stack.push(temp_stack.pop())


def browser_history_demo():
    from datetime import datetime, timedelta
    
    browser = BrowserHistory(max_history=10)
    
    # Simulate browsing
    print("SIMULATING BROWSER HISTORY")
    print("-" * 40)
    
    browser.visit("https://google.com", "Google")
    browser.visit("https://github.com", "GitHub")
    browser.visit("https://stackoverflow.com", "Stack Overflow")
    browser.visit("https://python.org", "Python.org")
    
    browser.display_history()
    
    # Go back
    print("\n=== Going back twice ===")
    browser.go_back()
    browser.go_back()
    browser.display_history()
    
    # Visit new page (should clear forward stack)
    print("\n=== Visiting new page ===")
    browser.visit("https://docs.python.org", "Python Documentation")
    browser.display_history()
    
    # Try forward (should have none due to new visit)
    print("\n=== Trying to go forward ===")
    browser.go_forward()
    
    # Go back and then forward
    print("\n=== Going back then forward ===")
    browser.go_back()
    browser.go_forward()
    browser.display_history()
    
    # Search history
    print("\n=== Searching history for 'python' ===")
    results = browser.search_history("python")
    if results:
        print("Search results:")
        for page in results:
            print(f"  - {page}")
    else:
        print("No results found")
    
    # Most visited
    print("\n=== Most visited pages ===")
    most_visited = browser.get_most_visited()
    for i, page in enumerate(most_visited, 1):
        print(f"{i}. {page} - Visited {page.visit_count} times")
    
    # Simulate more visits for statistics
    print("\n=== Simulating more visits ===")
    for _ in range(3):
        browser.visit("https://github.com", "GitHub")
    
    browser.visit("https://google.com", "Google")
    
    print("\n=== Updated most visited ===")
    most_visited = browser.get_most_visited()
    for i, page in enumerate(most_visited, 1):
        print(f"{i}. {page} - Visited {page.visit_count} times")


if __name__ == "__main__":
    browser_history_demo()